# Base Algorithms Performances

## Libraries

In [ ]:
import time
import os
os.chdir("../input")

import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

## Helper Functions

In [8]:
def train_and_evaluate_classifier(clf, X, y, cv=5):
    start = time.time()
    
    y_pred = cross_val_predict(clf, X, y, cv=cv)

    if hasattr(clf, "predict_proba"):
        y_proba = cross_val_predict(clf, X, y, cv=cv, method="predict_proba")[:, 1]
    else:
        y_proba = None

    end = time.time()
    elapsed_time = end - start

    metrics = {
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred, zero_division=0),
        'Recall': recall_score(y, y_pred, zero_division=0),
        'F1-Score': f1_score(y, y_pred, zero_division=0),
        'AUC-ROC': roc_auc_score(y, y_proba) if y_proba is not None else None,
        'AUC-PR': average_precision_score(y, y_proba) if y_proba is not None else None,
        'Training Time (s)': elapsed_time
    }

    return metrics

## Data Reading & Preprocessing

In [2]:
df = pd.read_csv("heart.csv")

In [ ]:
X = df.drop("output", axis=1)
y = df["output"]